In [4]:
import pandas as pd 
import json 
import os
from tqdm import tqdm
import subprocess
import platform
import threading
import jsonschema
from time import time


In [2]:
df = pd.read_csv('./results/results.csv')
df.head()

,schema,DG_ted,JE_ted,JSF_ted,DG_number_errors,JE_number_errors,JSF_number_errors,correct_instance_ast_size,DG_ast_size,JE_ast_size,JSF_ast_size
0,o1,0.0,14.0,60.0,0.0,0.0,0.0,3,3.0,17.0,73.0
1,o10008,0.0,116.0,1.0,0.0,0.0,0.0,5,5.0,155.0,5.0
2,o10009,0.0,0.0,8.0,0.0,0.0,0.0,1,1.0,1.0,9.0
3,o10010,0.0,0.0,4.0,0.0,0.0,0.0,1,1.0,1.0,5.0
4,o10011,0.0,2.0,10.0,0.0,2.0,0.0,3,3.0,3.0,15.0


In [3]:
df[df['JSF_number_errors']>10000]

,schema,DG_ted,JE_ted,JSF_ted,DG_number_errors,JE_number_errors,JSF_number_errors,correct_instance_ast_size,DG_ast_size,JE_ast_size,JSF_ast_size
2986,o63193,0.0,NaN,154308.0,0.0,NaN,148634.0,1,1.0,NaN,202371.0


In [14]:
CORRECT_INSTANCES_PATH='./dataset/correct-instances'
if not os.path.exists(CORRECT_INSTANCES_PATH): 
    os.mkdir(CORRECT_INSTANCES_PATH)

# 1- Processing data 

since the data is stringified, we need a way to save it into the correct format (json)

In [10]:
df = pd.read_csv("./dataset/01102022_150755_3600000ms_witness.csv")

In [11]:
df.head()

,objectId,witness
0,o27825,true
1,o27841,0
2,o67205,0
3,o89637,0
4,o27830,0


In [81]:
len(df.index)

6335

In [97]:
print(f"Processing : {df.index} correct instances")
for i in tqdm(df.index) :
    objectId,witness = df.loc[i]['objectId'],df.loc[i]['witness']
    if str(witness)=='nan' :
        witness = 'null'   
    # print(objectId,witness)
    json_witness = json.loads(witness)
    with open(f'{CORRECT_INSTANCES_PATH}/{objectId}.json','w',encoding='utf-8') as f : 
        f.write(json.dumps(json_witness))

Processing : RangeIndex(start=0, stop=6335, step=1) correct instances


100%|██████████| 6335/6335 [00:08<00:00, 728.66it/s] 


# 2- Transfering data to bracket notation

In [121]:
BRACKET_DIR = './dataset/bracket'
dirs = ['correct-instances','DG','JE','JSF']
BRACKET_SCRIPT_PATH ='./scripts/prepare.sh'

In [99]:
if not os.path.exists(BRACKET_DIR):
    os.makedirs(BRACKET_DIR)

In [126]:
for dir in dirs : 
    DESTINATION_DIR_BRACKET=f'./dataset/bracket/{dir}'
    if not os.path.exists(DESTINATION_DIR_BRACKET):
        os.makedirs(DESTINATION_DIR_BRACKET)

In [133]:
def process_bracket_directory(dir):
    for file in tqdm(os.listdir(f'./dataset/{dir}') ): 
        file_name = file.split('.')[0]
        
        SOURCE_JSON=f'./dataset/{dir}/{file_name}'
        DESTINATION_DIR_BRACKET=f'./dataset/bracket/{dir}'
        DESTINATION_BRACKET=f'{DESTINATION_DIR_BRACKET}/{file_name}'
        if platform.system() == 'Windows' : 
            output = subprocess.call(['wsl',BRACKET_SCRIPT_PATH,SOURCE_JSON,DESTINATION_BRACKET])
        else : 
            output = subprocess.call([BRACKET_SCRIPT_PATH,SOURCE_JSON,DESTINATION_BRACKET])

In [135]:
threads = []
for dir in dirs:
    thread = threading.Thread(target=process_bracket_directory, args=(dir))
    thread.start()
    threads.append(thread)

# Wait for all threads to finish
for thread in threads:
    thread.join()
print("All directories processed.")

  0%|          | 0/6335 [00:00<?, ?it/s]





  0%|          | 1/6335 [00:01<2:29:33,  1.42s/it]


  0%|          | 2/6335 [00:02<1:55:13,  1.09s/it]

  0%|          | 3/6335 [00:03<1:51:16,  1.05s/it]



  0%|          | 4/6335 [00:05<2:18:32,  1.31s/it]


  0%|          | 5/6335 [00:06<2:07:50,  1.21s/it]


  0%|          | 6/6335 [00:07<2:20:10,  1.33s/it]


  0%|          | 8/6335 [00:09<1:55:18,  1.09s/it]


  0%|          | 9/6335 [00:10<1:49:05,  1.03s/it]


  0%|          | 10/6335 [00:11<1:46:13,  1.01s/it]


  0%|          | 11/6335 [00:12<1:41:22,  1.04it/s]




  0%|          | 12/6335 [00:13<1:46:03,  1.01s/it]


  0%|          | 13/6335 [00:15<2:21:29,  1.34s/it]


  0%|          | 14/6335 [00:16<2:15:54,  1.29s/it]


  0%|          | 15/6335 [00:18<2:38:50,  1.51s/it]


  0%|          | 16/6335 [00:20<2:39:55,  1.52s/it]


  0%|          | 17/6335 [00:20<2:20:53,  1.34s/it]


  0%|          | 18/6335 [00:21<2:11:21,  1.25s/it]


  0%|          | 19/6335 [00:22<1:56:45,  1.11s/it]


  0%|          | 20/6335 [00:24<2

All directories processed.


# 3- Comparing edit distances 

## 3.1 Define directories

In [15]:
SCHEMAS_PATH = './dataset/schemas'
CORRECT_INSTANCES_BRACKET_PATH = './dataset/bracket/correct-instances'
EXPERIMENT_SCRIPT = './scripts/expr.sh'
RESULTS_FILE = './results/results.csv'


In [16]:
witnesses = ['DG','JE','JSF']
witnesses_edit_distances_columns = [f'{x}_ted'for x in witnesses ]
witnesses_number_errors_columns = [f'{x}_number_errors'for x in witnesses ]


# Witness dirs + bracket 
witnesses_dirs = [f'./dataset/{x}'for x in witnesses ]
witnesses_bracket_dirs = [f'./dataset/bracket/{x}'for x in witnesses ]

# AST node stuff 
witnesses_ast_sizes_columns = [f'{x}_ast_size'for x in witnesses ]
ast_sizes_columns = ['correct_instance_ast_size']+witnesses_ast_sizes_columns
ast_sizes_dirs = [CORRECT_INSTANCES_PATH]+witnesses_dirs

columns = ['schema']+ witnesses_edit_distances_columns+witnesses_number_errors_columns+ast_sizes_columns

## 3.2 Initialize the dataframe

In [17]:
def initialise_results(columns):
    df = pd.DataFrame(columns=columns)
    if not os.path.exists('./results'):
        os.makedirs('./results')
    df.to_csv('./results/results.csv',index=False)

In [18]:
def witness_file_name(schema_name,is_json_file=False):
    return schema_name+'_witness.'+ ('json' if is_json_file else 'bracket')

## 3.3 Calculating edit distance

In [21]:
def calculate_edit_distance(schema_name:str,correct_instance_schema_bracket_path:str,w_bracket_dir)->int | None: 
    witness_bracket_path = f'{w_bracket_dir}/{witness_file_name(schema_name)}'
    if not os.path.exists(witness_bracket_path) : 
        return None 
    # print(correct_instance_schema_bracket_path)
    # print(witness_bracket_path)
    if platform.system() == 'Windows':
        command=['wsl',EXPERIMENT_SCRIPT,correct_instance_schema_bracket_path,witness_bracket_path]
    else : 
        command = [EXPERIMENT_SCRIPT,correct_instance_schema_bracket_path,witness_bracket_path]
    max_retries = 3
    retry_delay = 1  # seconds
    
    for retry in range(max_retries):
        try:
            process = subprocess.run(command, capture_output=True, text=True, timeout=30)  # Increase or decrease timeout as needed
            if process.returncode != 0:
                # print("Subprocess failed:", process.stderr)
                continue
            output_string = process.stdout.strip()
            output_parts = output_string.split(':')
            if len(output_parts) < 2:
                # print("Unexpected output format. Expected 'TED Distances: <Number>'. Actual:", output_string)
                continue 
            result = int(float(output_parts[1]))
            return result
        except subprocess.TimeoutExpired:
            print(f"Subprocess timed out on attempt {retry + 1}. Retrying...")
            # time.sleep(retry_delay)
    
    # print("Max retries exceeded. Unable to calculate edit distance.")
    return None
  
# print(calculate_edit_distance('o9915','./dataset/bracket/correct-instances/o9915.bracket','./dataset/bracket/JE/'))

## 3.4 Calculating the number of errors 

In [22]:
def calcualte_number_errors(schema_name,witness_path):
    with open(f'{SCHEMAS_PATH}/{schema_name}.json','r',encoding='utf-8') as f : 
        schema = json.load(f)
    if not os.path.exists(witness_path):
        return None 
    try : 
        with open(witness_path,'r',encoding='utf-8') as f : 
            data = json.load(f)
    except :
        return None
    try : 
        validator = jsonschema.Draft7Validator(schema)
        # print(list(validator.iter_errors(data)))
        return len(list(validator.iter_errors(data)))
    except Exception as e : 
        if("error: nothing to repeat at position 1" in str(e)) : # error appearing somehow
            return 0
        else :
            return None 
        
    
# calcualte_number_errors('o33194',f'./dataset/JE/{witness_file_name("o33194",True)}')

## 3.5 Calculate the number of nodes of a json schema 

In [36]:

def get_size_json(doc):
    """Returns the size of the corresponding tree of the given JSON."""

    size = 0
    if isinstance(doc, dict): # OBJECT
        # Count the node of the object and all its keys.
        size += 1 + len(doc.keys())
        # Add the sizes of all values.
        for key, val in doc.items():
            size += get_size_json(val)
    elif isinstance(doc, list): # ARRAY
        # Count the node of the array and all its array order nodes.
        size += 1 + len(doc)
        # Add the sizes of all values.
        for val in doc:
            size += get_size_json(val)
    else: # VALUE
        # Add node for the value.
        size += 1
    return size
def calculate_ast_size(json_file_path:str):
    try : 
        with open(json_file_path) as json_file:
            data = json.load(json_file)
            tree_size  = get_size_json(data)
        return tree_size 
    except :
        return None

In [25]:
def save_results(schema_name:str,results:list,columns:list):
    df = pd.DataFrame([[schema_name]+results],columns=columns) 
    df.to_csv(RESULTS_FILE,mode='a',index=False,header=False)

In [38]:
def process_schema_edit_distance_with_generators(schema:str):
    schema_name = schema.split('.')[0]
    correct_instance_schema_bracket_path = f'{CORRECT_INSTANCES_BRACKET_PATH}/{schema_name}.bracket'
    results = []
    # calculate the edit distance 
    print(f"Processing {schema_name}")
    print("Calculating edit distance : ")
    for w_gen,w_bracket_dir in zip(witnesses,witnesses_bracket_dirs): 
        # print(f"\t >{w_gen}")
        result = calculate_edit_distance(schema_name,correct_instance_schema_bracket_path,w_bracket_dir)
        results.append(result)
    # print(results)
    if all([x== None for x in results]):
        return 
    print("Calculating errors")
    # calculate the number of errors  
    for w_gen,w_dir in zip(witnesses,witnesses_dirs): 
        # print(f"\t >{w_gen}")
        w_path = f'{w_dir}/{witness_file_name(schema_name,True)}'
        result = calcualte_number_errors(schema_name,w_path)
        results.append(result)
    
    # calculate the number of ast 
    print("Caluculating number of ast nodes")
    for i,(instance_column_name,instance_base_dir) in enumerate(zip(ast_sizes_columns,ast_sizes_dirs)):
        file_name = schema if i==0 else witness_file_name(schema_name,True)
        json_instance_path = f'{instance_base_dir}/{file_name}'
        result = calculate_ast_size(json_instance_path)
        results.append(result)
    
    print(f"Processed {schema_name}")
    save_results(schema_name,results,columns)
    pass  
    

In [39]:
initialise_results(columns)
for schema in tqdm(os.listdir(SCHEMAS_PATH)):
    process_schema_edit_distance_with_generators(schema)
    


  0%|          | 0/6377 [00:00<?, ?it/s]

Processing o1
Calculating edit distance : 


  0%|          | 1/6377 [00:00<1:03:58,  1.66it/s]

Calculating errors
Caluculating number of ast nodes
Processed o1
Processing o10008
Calculating edit distance : 


  0%|          | 2/6377 [00:01<1:01:54,  1.72it/s]

...

100%|█████████▉| 6375/6377 [1:48:28<00:01,  1.16it/s]

Calculating errors
Caluculating number of ast nodes
Processed o9972
Processing o9976
Calculating edit distance : 


100%|█████████▉| 6376/6377 [1:48:29<00:00,  1.07it/s]

Calculating errors
Caluculating number of ast nodes
Processed o9976
Processing o9981
Calculating edit distance : 
Calculating errors
Caluculating number of ast nodes
Processed o9981


100%|██████████| 6377/6377 [1:48:31<00:00,  1.02s/it]


In [338]:
"Distance TED:128".split(':')[1]

'128'

2


0